# EDA for HMDA in Orange County, NC, 2018, 2019


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, RandomizedSearchCV, KFold
from sklearn.preprocessing import StandardScaler

In [44]:
def merge_data():
    '''
    merge 2018, 19 data and write them to a new csv file
    '''
    Orange_2018 = pd.read_csv("./data_raw/county_37135_2018.csv")
    Orange_2019 = pd.read_csv("./data_raw/county_37135_2019.csv")
    Orange = pd.concat([Orange_2018, Orange_2019])
    Orange.to_csv("./data_cleaned/county_37135.csv", index=False)

# merge_data()

In [49]:
# Orange = pd.read_csv("./data_cleaned/county_37135.csv")


64

In [50]:
Orange.columns[Orange.isnull().mean() < 0.2]

Index([&#39;Unnamed: 0&#39;, &#39;activity_year&#39;, &#39;lei&#39;, &#39;derived_msa-md&#39;, &#39;state_code&#39;,
       &#39;county_code&#39;, &#39;census_tract&#39;, &#39;conforming_loan_limit&#39;,
       &#39;derived_loan_product_type&#39;, &#39;derived_dwelling_category&#39;,
       &#39;derived_ethnicity&#39;, &#39;derived_race&#39;, &#39;derived_sex&#39;, &#39;action_taken&#39;,
       &#39;purchaser_type&#39;, &#39;preapproval&#39;, &#39;loan_type&#39;, &#39;loan_purpose&#39;,
       &#39;lien_status&#39;, &#39;reverse_mortgage&#39;, &#39;open-end_line_of_credit&#39;,
       &#39;business_or_commercial_purpose&#39;, &#39;loan_amount&#39;, &#39;hoepa_status&#39;,
       &#39;loan_term&#39;, &#39;negative_amortization&#39;, &#39;interest_only_payment&#39;,
       &#39;balloon_payment&#39;, &#39;other_nonamortizing_features&#39;, &#39;property_value&#39;,
       &#39;construction_method&#39;, &#39;occupancy_type&#39;,
       &#39;manufactured_home_secured_property_type&#39;,
  

In [55]:
def drop_na_columns():
    merge_data()
    Orange = pd.read_csv("./data_cleaned/county_37135.csv")
    Orange.replace(r'\s+',np.nan,regex=True).replace('',np.nan).replace('NA', np.nan)
    # drop all columns that has too much (> 20%) missing data
    Orange = Orange[Orange.columns[Orange.isnull().mean() < 0.1]].iloc[:, 1: ]
    Orange.to_csv("./data_cleaned/county_37135.csv", index=False)
# drop_na_columns()

In [71]:
Orange = pd.read_csv("./data_cleaned/county_37135.csv")
cols = [c for c in Orange.columns if not (c.startswith("applicant_ethnicity") or c.startswith("co-applicant_ethnicity") or c.endswith("observed"))]
cols

[&#39;activity_year&#39;,
 &#39;lei&#39;,
 &#39;derived_msa-md&#39;,
 &#39;state_code&#39;,
 &#39;county_code&#39;,
 &#39;census_tract&#39;,
 &#39;conforming_loan_limit&#39;,
 &#39;derived_loan_product_type&#39;,
 &#39;derived_dwelling_category&#39;,
 &#39;derived_ethnicity&#39;,
 &#39;derived_race&#39;,
 &#39;derived_sex&#39;,
 &#39;action_taken&#39;,
 &#39;purchaser_type&#39;,
 &#39;preapproval&#39;,
 &#39;loan_type&#39;,
 &#39;loan_purpose&#39;,
 &#39;lien_status&#39;,
 &#39;reverse_mortgage&#39;,
 &#39;open-end_line_of_credit&#39;,
 &#39;business_or_commercial_purpose&#39;,
 &#39;loan_amount&#39;,
 &#39;hoepa_status&#39;,
 &#39;loan_term&#39;,
 &#39;negative_amortization&#39;,
 &#39;interest_only_payment&#39;,
 &#39;balloon_payment&#39;,
 &#39;other_nonamortizing_features&#39;,
 &#39;construction_method&#39;,
 &#39;occupancy_type&#39;,
 &#39;manufactured_home_secured_property_type&#39;,
 &#39;manufactured_home_land_property_interest&#39;,
 &#39;total_units&#39;,
 &#39;income&#39;,


In [64]:
Orange.columns

Index([&#39;activity_year&#39;, &#39;lei&#39;, &#39;derived_msa-md&#39;, &#39;state_code&#39;, &#39;county_code&#39;,
       &#39;census_tract&#39;, &#39;conforming_loan_limit&#39;, &#39;derived_loan_product_type&#39;,
       &#39;derived_dwelling_category&#39;, &#39;derived_ethnicity&#39;, &#39;derived_race&#39;,
       &#39;derived_sex&#39;, &#39;action_taken&#39;, &#39;purchaser_type&#39;, &#39;preapproval&#39;,
       &#39;loan_type&#39;, &#39;loan_purpose&#39;, &#39;lien_status&#39;, &#39;reverse_mortgage&#39;,
       &#39;open-end_line_of_credit&#39;, &#39;business_or_commercial_purpose&#39;,
       &#39;loan_amount&#39;, &#39;hoepa_status&#39;, &#39;loan_term&#39;, &#39;negative_amortization&#39;,
       &#39;interest_only_payment&#39;, &#39;balloon_payment&#39;,
       &#39;other_nonamortizing_features&#39;, &#39;construction_method&#39;, &#39;occupancy_type&#39;,
       &#39;manufactured_home_secured_property_type&#39;,
       &#39;manufactured_home_land_property_interest&#39;

In [5]:
NC_data = pd.read_csv("data_raw/hmda_2013_nc_all-records_labels.csv")


Index([&#39;as_of_year&#39;, &#39;respondent_id&#39;, &#39;agency_name&#39;, &#39;agency_abbr&#39;,
       &#39;agency_code&#39;, &#39;loan_type_name&#39;, &#39;loan_type&#39;, &#39;property_type_name&#39;,
       &#39;property_type&#39;, &#39;loan_purpose_name&#39;, &#39;loan_purpose&#39;,
       &#39;owner_occupancy_name&#39;, &#39;owner_occupancy&#39;, &#39;loan_amount_000s&#39;,
       &#39;preapproval_name&#39;, &#39;preapproval&#39;, &#39;action_taken_name&#39;, &#39;action_taken&#39;,
       &#39;msamd_name&#39;, &#39;msamd&#39;, &#39;state_name&#39;, &#39;state_abbr&#39;, &#39;state_code&#39;,
       &#39;county_name&#39;, &#39;county_code&#39;, &#39;census_tract_number&#39;,
       &#39;applicant_ethnicity_name&#39;, &#39;applicant_ethnicity&#39;,
       &#39;co_applicant_ethnicity_name&#39;, &#39;co_applicant_ethnicity&#39;,
       &#39;applicant_race_name_1&#39;, &#39;applicant_race_1&#39;, &#39;applicant_race_name_2&#39;,
       &#39;applicant_race_2&#39;, &#39;applicant_ra

In [6]:
len(NC_data.columns)

78